In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.font_manager as fm
import matplotlib
import timeit
import feather
%matplotlib inline

#### Step 1. Prepare datafram to compare

#### 1. Get the dataframe that has shape_id, route, sequence of stops and stopID base on the "googletransit20130315-1546/stop_times.csv" data

In [2]:
# Read in stop_times.csv and trips.csv
stop_times_2012 = pd.read_csv('stop_times.csv')
trips_2012 = pd.read_csv('trips.csv')

# Merge by trip_id
# trip_id here has the same meaning of JourneyPatternID
merge = pd.merge(stop_times_2012, trips_2012, on='trip_id', how='outer')

# Keep only necessary columns
merge.drop(['arrival_time','departure_time','pickup_type','drop_off_type','service_id','shape_dist_traveled'], axis=1, inplace=True)

In [3]:
# Seporate trip_id to route, stop_id to StopID
# Route
merge['route_short'] = merge['trip_id'].apply(lambda x: x[x.index('-')+1:])
merge['route_short'] = merge['route_short'].apply(lambda x: x[: x.index('-')])
merge['route_short'] = merge['route_short'].apply(lambda x: str(x).zfill(4))

# StopID
merge['StopID'] = merge['stop_id'].apply(lambda x: x[-4:])

In [4]:
# Find StopID and the sequence of that shape_id
gb = merge.groupby(['shape_id', 'route_short', 'direction_id','stop_sequence', 'StopID'])
gbc = gb.count()
gbc.reset_index(['shape_id', 'route_short', 'direction_id','stop_sequence', 'StopID'], inplace=True)
transit_shapeID_stopID = gbc.drop(['trip_id','stop_id','route_id','trip_headsign'], axis=1)

In [5]:
# Now we have trip_id to its corresponding StopID and sequence
transit_shapeID_stopID

,shape_id,route_short,direction_id,stop_sequence,StopID
0,0-1-y12-1.1.O,0001,0,1,0226
1,0-1-y12-1.1.O,0001,0,2,0228
2,0-1-y12-1.1.O,0001,0,3,0229
3,0-1-y12-1.1.O,0001,0,4,0227
4,0-1-y12-1.1.O,0001,0,5,0230
5,0-1-y12-1.1.O,0001,0,6,0231
6,0-1-y12-1.1.O,0001,0,7,1641
7,0-1-y12-1.1.O,0001,0,8,1642
8,0-1-y12-1.1.O,0001,0,9,0213
9,0-1-y12-1.1.O,0001,0,10,0214


In [6]:
ssid = []
for i in range(len(transit_shapeID_stopID.index)-1):
    temp = transit_shapeID_stopID['StopID'].iloc[i] + transit_shapeID_stopID['StopID'].iloc[i+1]
    ssid.append([ transit_shapeID_stopID['route_short'].iloc[i],temp])
print(ssid)

[['0001', '02260228'], ['0001', '02280229'], ['0001', '02290227'], ['0001', '02270230'], ['0001', '02300231'], ['0001', '02311641'], ['0001', '16411642'], ['0001', '16420213'], ['0001', '02130214'], ['0001', '02144432'], ['0001', '44320119'], ['0001', '01190044'], ['0001', '00440045'], ['0001', '00450046'], ['0001', '00460047'], ['0001', '00470048'], ['0001', '00480049'], ['0001', '00490050'], ['0001', '00500051'], ['0001', '00510052'], ['0001', '00520265'], ['0001', '02650271'], ['0001', '02710340'], ['0001', '03400350'], ['0001', '03500351'], ['0001', '03510352'], ['0001', '03520353'], ['0001', '03530354'], ['0001', '03540355'], ['0001', '03550356'], ['0001', '03560357'], ['0001', '03570390'], ['0001', '03900372'], ['0001', '03720373'], ['0001', '03730374'], ['0001', '03740375'], ['0001', '03752804'], ['0001', '28040376'], ['0001', '03760377'], ['0001', '03770378'], ['0001', '03780380'], ['0001', '03800265'], ['0001', '02650271'], ['0001', '02710340'], ['0001', '03400350'], ['0001', 

In [19]:
test_routes = [x for x in ssid if '09090786' in x[1]]
test_routes = [item[0] for item in test_routes]
test_routes = list(set(test_routes))
print (test_routes)

['084X', '007D', '007B', '0011', '032X', '0116', '0145', '046E', '041X', '0118', '046A']


In [22]:
test_routes = [x for x in ssid if 'YOUR_NUMBER_HERE' in x[1]]
test_routes = [item[0] for item in test_routes]
test_routes = list(set(test_routes))
print (test_routes)

[]
